In [3]:
from src.models.base import init_db, get_session
from src.models.player import Player
from pathlib import Path
from src.agents.dm_orchestrator import DMOrchestrator

In [5]:
import os
db_path = os.path.abspath("../data/sw.db")
print(f"Using database: {db_path}")  # Verify the path
init_db(db_path)

Using database: C:\Users\Dell\PycharmProjects\llm_gm\agent_gm\data\sw.db


In [6]:
with get_session() as session:
    # Check for existing player
    players = session.query(Player).all()
    print(f"Found {len(players)} players in the database.")
    print(players)
    print(players[0])
    player = players[0] if players else None
    
    player_id = player.id if player else None
    
    

Found 2 players in the database.
[<Player(id=c78b34d5-4177-4452-a779-633617fcec92, name=Mathias Thun)>, <Player(id=8750c701-4e24-434f-9404-011f64a2337c, name=Probuvach)>]
<Player(id=c78b34d5-4177-4452-a779-633617fcec92, name=Mathias Thun)>


In [7]:
dm = DMOrchestrator(player_id=player_id)

Session message history has an orphaned toolUse with no toolResult. Adding toolResult content blocks to create valid conversation.


In [8]:
state = dm.agent.state

In [9]:
state

In [10]:
# Test getting past messages from the DM session using FileSessionManager
import os
import tempfile
from strands.session.file_session_manager import FileSessionManager
from strands_semantic_memory.message_utils import extract_text_content

# The DM uses player_id as session_id and "default" as agent_id
storage_dir = os.path.join(tempfile.gettempdir(), "strands/sessions")
print(f"Storage dir: {storage_dir}")
print(f"Player ID: {player_id}")

Storage dir: C:\Users\Dell\AppData\Local\Temp\strands/sessions
Player ID: c78b34d5-4177-4452-a779-633617fcec92


In [11]:
# Create a session manager to read the DM's messages
session_manager = FileSessionManager(session_id=player_id, storage_dir=storage_dir)

# List all messages for the DM agent (agent_id="default")
all_messages = session_manager.list_messages(
    session_id=player_id,
    agent_id="default",
)

print(f"Found {len(all_messages)} messages in DM session")

Found 445 messages in DM session


In [12]:
# Get the last 6 messages
num_messages = 6
recent_messages = all_messages[-num_messages:] if len(all_messages) > num_messages else all_messages

# Inspect the raw message structure
for i, session_msg in enumerate(recent_messages):
    message = session_msg.to_message()
    print(f"\n=== Message {i} ===")
    print(f"Role: {message.get('role')}")
    print(f"Content types: {[type(c).__name__ if not isinstance(c, dict) else list(c.keys()) for c in message.get('content', [])]}")


=== Message 0 ===
Role: user
Content types: [['text']]

=== Message 1 ===
Role: assistant
Content types: [['reasoningContent'], ['toolUse']]

=== Message 2 ===
Role: user
Content types: [['toolResult']]

=== Message 3 ===
Role: assistant
Content types: [['reasoningContent'], ['text']]

=== Message 4 ===
Role: user
Content types: [['text']]

=== Message 5 ===
Role: assistant
Content types: [['reasoningContent'], ['text']]


In [13]:
# Extract just the text content (no tool calls) and format it
context_parts = []
for session_msg in recent_messages:
    message = session_msg.to_message()
    role = message.get("role", "unknown")
    text = extract_text_content(message)
    
    if text.strip():
        if role == "user":
            context_parts.append(f"Player: {text.strip()}")
        else:
            context_parts.append(f"Narrator: {text.strip()}")

# Format as context string
context_string = "Recent conversation:\n" + "\n".join(context_parts)
print(context_string)

Recent conversation:
Player: Current context:
- Location: Jedi Temple Training Dojo (room)
- Time: Day 1, 8:00 (morning)
- NPCs here: Ahsoka Tano, Youngling Initiate, Jedi Master Cin Drallig

Player says: (You aren't outputting anything)
Narrator: The simulation begins around you. Holographic battle droids materialize in a semicircle, their red photoreceptors locking onto your position. Behind them, the distinctive forms of B2 super battle droids take position, their heavy arm cannons raising to target you. E-Web repeating blasters spin up on elevated positions, and you can see the agile silhouettes of BX-series commando droids moving to flank you.

You're facing exactly what Captain Gears described - layered defenses, overwhelming numbers, elite droids that know how to kill Jedi.

Ahsoka watches from the observation window, her montrals focused on your every move. She can sense your focus, your preparation. She wants to see how you'll handle this - how a Jedi Knight trained in Form 2 

In [14]:
# Wrap it into a reusable function
def get_recent_dm_context(player_id: str, num_messages: int = 6) -> str:
    """Get recent conversation context from the DM agent's session."""
    try:
        storage_dir = os.path.join(tempfile.gettempdir(), "strands/sessions")
        session_manager = FileSessionManager(session_id=player_id, storage_dir=storage_dir)
        
        all_messages = session_manager.list_messages(
            session_id=player_id,
            agent_id="default",
        )
        
        if not all_messages:
            return ""
        
        recent_messages = all_messages[-num_messages:] if len(all_messages) > num_messages else all_messages
        
        context_parts = []
        for session_msg in recent_messages:
            message = session_msg.to_message()
            role = message.get("role", "unknown")
            text = extract_text_content(message)
            
            if text.strip():
                if role == "user":
                    context_parts.append(f"Player: {text.strip()}")
                else:
                    context_parts.append(f"Narrator: {text.strip()}")
        
        if not context_parts:
            return ""
        
        return "Recent conversation:\n" + "\n".join(context_parts)
    except Exception as e:
        print(f"Error: {e}")
        return ""

# Test the function
result = get_recent_dm_context(player_id)
print("=== Function Output ===")
print(result)

=== Function Output ===
Recent conversation:
Player: Current context:
- Location: Jedi Temple Training Dojo (room)
- Time: Day 1, 8:00 (morning)
- NPCs here: Ahsoka Tano, Youngling Initiate, Jedi Master Cin Drallig

Player says: (You aren't outputting anything)
Narrator: The simulation begins around you. Holographic battle droids materialize in a semicircle, their red photoreceptors locking onto your position. Behind them, the distinctive forms of B2 super battle droids take position, their heavy arm cannons raising to target you. E-Web repeating blasters spin up on elevated positions, and you can see the agile silhouettes of BX-series commando droids moving to flank you.

You're facing exactly what Captain Gears described - layered defenses, overwhelming numbers, elite droids that know how to kill Jedi.

Ahsoka watches from the observation window, her montrals focused on your every move. She can sense your focus, your preparation. She wants to see how you'll handle this - how a Jedi K